In [1]:
from bs4 import BeautifulSoup
import requests
import urllib, json
import re
import pandas as pd

In [2]:
def Category():
    url="https://ekantipur.com"
    r=requests.get(url)
    html_file=r.text
    soup=BeautifulSoup(html_file)
    label=soup.findAll('li')[2:25]
    
    links=[]
    category=[]
    for div in soup.findAll('li')[2:25]:
        
        a = div.findAll('a')[0]
        print(a)
        link=a.attrs['href']
        links.append(link)
    for i in links:
        removed = i.replace("https://ekantipur.com/", "")
        category.append(removed)
    return category
Category()

<a href="https://ekantipur.com/news">समाचार</a>
<a href="https://ekantipur.com/business">अर्थ / वाणिज्य</a>
<a href="https://ekantipur.com/opinion">विचार</a>
<a href="https://ekantipur.com/sports">खेलकुद</a>
<a href="https://ekantipur.com/national">उपत्यका</a>
<a href="https://ekantipur.com/koseli">कोसेली</a>
<a href="https://ekantipur.com/world">विश्व</a>
<a href="https://ekantipur.com/entertainment">मनोरञ्जन</a>
<a href="https://ekantipur.com/blog">ब्लग</a>
<a href="https://ekantipur.com/diaspora">प्रवास</a>
<a href="https://ekantipur.com/feature">फिचर</a>
<a href="https://ekantipur.com/photo_feature">फोटोफिचर</a>
<a href="https://ekantipur.com/lifestyle">जीवनशैली</a>
<a href="https://ekantipur.com/literature">साहित्य/विविध</a>
<a href="https://ekantipur.com/technology">विज्ञान र प्रविधि</a>
<a href="https://ekantipur.com/health">स्वास्थ्य</a>
<a href="https://ekantipur.com/video">भिडियो</a>
<a href="https://ekantipur.com/pathakmanch">सम्पादकलाई चिठी</a>
<a href="https://ekantipur.co

['news',
 'business',
 'opinion',
 'sports',
 'national',
 'koseli',
 'world',
 'entertainment',
 'blog',
 'diaspora',
 'feature',
 'photo_feature',
 'lifestyle',
 'literature',
 'technology',
 'health',
 'video',
 'pathakmanch',
 'Interview',
 'Art',
 'Other',
 'nari-nepali',
 'DASHAIN-ESPICAL']

In [3]:
def Link_extract(category):
    category_url='https://ekantipur.com/'+category+''
    r=requests.get(category_url)
    sporthtml=r.text
    soup=BeautifulSoup(sporthtml,'html.parser')
    label=soup.findAll("div", {"class": "catName"})
    links=[]
    for i in label:
        for j in i:
            label=j
    for div in soup.findAll("div", {"class": "teaser offset"}):
        a = div.findAll('a')[0]
        link=a.attrs['href']
        links.append("https://ekantipur.com"+link)
    return links,label


In [4]:
def ajax_link(category):
    ajax_link=[]
    links,_=Link_extract(category)
    for i in range(1,29,2):
        my_url = 'https://ekantipur.com/'+category+'/2019/10/'+str(i)+'?json=true' 
        ret = requests.get(my_url)
        response = urllib.request.urlopen(my_url)
        data = json.loads(response.read())
        data=data['html']
        urls = re.findall('(?:(?:https?|ftp)://ekantipur.com/'+category+'/)[\w/\-?=%.]+\.[\w/\-?=%.]+', data)

        for i in urls:
            ajax_link.append(i)
    for s in list(set(ajax_link)):
        links.append(s)
    return links


In [5]:
def Get_data(category):
    final_data=[]
    data=[]
    links=ajax_link(category)
    label=[]
    for i in range(len(links)):
        soup = BeautifulSoup(requests.get(links[i]).text,"html.parser")
        for item in soup.findAll("div", class_=['description', 'portrait']):
            if item['class'][0]=='description':
                required_data = [p_item.text.strip() for p_item in item.select("p")]
                required_data=required_data[0:2]
                for a in required_data:
                    data.append(a)      
        final_data.append(" ".join(data[0:2]))
        data=[]
        for item in  soup.select('.cat_name div:first-child'):
            required_data = [p_item.text.strip() for p_item in item.select("a")]
            for i in required_data:
                if len(i)==0:
                    pass
                else:
                    label.append(i) 
        
    return  final_data,label


In [6]:
len(Category())

<a href="https://ekantipur.com/news">समाचार</a>
<a href="https://ekantipur.com/business">अर्थ / वाणिज्य</a>
<a href="https://ekantipur.com/opinion">विचार</a>
<a href="https://ekantipur.com/sports">खेलकुद</a>
<a href="https://ekantipur.com/national">उपत्यका</a>
<a href="https://ekantipur.com/koseli">कोसेली</a>
<a href="https://ekantipur.com/world">विश्व</a>
<a href="https://ekantipur.com/entertainment">मनोरञ्जन</a>
<a href="https://ekantipur.com/blog">ब्लग</a>
<a href="https://ekantipur.com/diaspora">प्रवास</a>
<a href="https://ekantipur.com/feature">फिचर</a>
<a href="https://ekantipur.com/photo_feature">फोटोफिचर</a>
<a href="https://ekantipur.com/lifestyle">जीवनशैली</a>
<a href="https://ekantipur.com/literature">साहित्य/विविध</a>
<a href="https://ekantipur.com/technology">विज्ञान र प्रविधि</a>
<a href="https://ekantipur.com/health">स्वास्थ्य</a>
<a href="https://ekantipur.com/video">भिडियो</a>
<a href="https://ekantipur.com/pathakmanch">सम्पादकलाई चिठी</a>
<a href="https://ekantipur.co

23

In [7]:
category=Category()
category=category
category

<a href="https://ekantipur.com/news">समाचार</a>
<a href="https://ekantipur.com/business">अर्थ / वाणिज्य</a>
<a href="https://ekantipur.com/opinion">विचार</a>
<a href="https://ekantipur.com/sports">खेलकुद</a>
<a href="https://ekantipur.com/national">उपत्यका</a>
<a href="https://ekantipur.com/koseli">कोसेली</a>
<a href="https://ekantipur.com/world">विश्व</a>
<a href="https://ekantipur.com/entertainment">मनोरञ्जन</a>
<a href="https://ekantipur.com/blog">ब्लग</a>
<a href="https://ekantipur.com/diaspora">प्रवास</a>
<a href="https://ekantipur.com/feature">फिचर</a>
<a href="https://ekantipur.com/photo_feature">फोटोफिचर</a>
<a href="https://ekantipur.com/lifestyle">जीवनशैली</a>
<a href="https://ekantipur.com/literature">साहित्य/विविध</a>
<a href="https://ekantipur.com/technology">विज्ञान र प्रविधि</a>
<a href="https://ekantipur.com/health">स्वास्थ्य</a>
<a href="https://ekantipur.com/video">भिडियो</a>
<a href="https://ekantipur.com/pathakmanch">सम्पादकलाई चिठी</a>
<a href="https://ekantipur.co

['news',
 'business',
 'opinion',
 'sports',
 'national',
 'koseli',
 'world',
 'entertainment',
 'blog',
 'diaspora',
 'feature',
 'photo_feature',
 'lifestyle',
 'literature',
 'technology',
 'health',
 'video',
 'pathakmanch',
 'Interview',
 'Art',
 'Other',
 'nari-nepali',
 'DASHAIN-ESPICAL']

In [ ]:
data=[]
label=[]
for i in category[0:3]:
    a,b=Get_data(i)
    for s in a:
        data.append(s)
    for k in b:
        label.append(k)

In [10]:
df_basic={'news':data,
  'label':label}

In [11]:
aa=pd.DataFrame(df_basic)

In [11]:
aa.to_csv('all.csv')

In [12]:
df_basic

{'news': [], 'label': []}